In [1]:
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import math
import time

import numpy as np
import tensorflow as tf

import cifar10_ACOL as cifar10

labelNames=np.array(['airplane','automobile','bird','cat','deer','dog','frog','horse', 'ship','truck'])
testSize=128*20
clustCount = 10
classCount = 2
#classNames = np.array(['Stuff and Horse','Animals'])
classNames = np.array(['airplane&bird','automobile&truck''cat&dog','deer&horse','frog&ship'])
supLab = tf.constant([0,1,0,3,4,3,5,4,5,1]) 
#supLab = tf.constant([0,0,1,1,1,1,1,0,0,0])

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import math
import time

import numpy as np
import tensorflow as tf

import cifar10_ACOL as cifar10

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('eval_dir', '/code/logs/cifar10_eval',
                           """Directory where to write event logs.""")
tf.app.flags.DEFINE_string('eval_data', 'test',
                           """Either 'test' or 'train_eval'.""")
tf.app.flags.DEFINE_string('checkpoint_dir', '/code/logs/cifar10_train',
                           """Directory where to read model checkpoints.""")
tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
                            """How often to run the eval.""")
tf.app.flags.DEFINE_integer('num_examples', 10000,
                            """Number of examples to run.""")
tf.app.flags.DEFINE_boolean('run_once', True,
                         """Whether to run eval only once.""")

def eval_once(saver, summary_writer, summary_op, softmaxmat, images, labels, img_raw, superLabels):
  """Run Eval once.

  Args:
    saver: Saver.
    summary_writer: Summary writer.
    top_k_op: Top K op.
    summary_op: Summary op.
  """
  with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
      # Restores from checkpoint
      #ckpt.model_checkpoint_path = "/code/logs/cifar10_train/model.ckpt-647452"
      saver.restore(sess, ckpt.model_checkpoint_path)
      print(ckpt.model_checkpoint_path)
      # Assuming model_checkpoint_path looks something like:
      #   /my-favorite-path/cifar10_train/model.ckpt-0,
      # extract global_step from it.
      global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
    else:
      print('No checkpoint file found')
      return

    # Start the queue runners.
    coord = tf.train.Coordinator()
    try:
      threads = []
      for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
        threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                         start=True))

      num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
      true_count = 0  # Counts the number of correct predictions.
      total_sample_count = num_iter * FLAGS.batch_size
      step = 0
      while step < num_iter and not coord.should_stop():
        softmaxmat_r, images_r, labels_r, img_raw_r, superLabels_r = sess.run([softmaxmat, images, labels, img_raw, superLabels])
        step += 1

      # Compute precision @ 1.
      #precision = true_count / total_sample_count
      #print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))

      #summary = tf.Summary()
      #summary.ParseFromString(sess.run(summary_op))
      #summary.value.add(tag='Precision @ 1', simple_value=precision)
      #summary_writer.add_summary(summary, global_step)
    except Exception as e:  # pylint: disable=broad-except
      coord.request_stop(e)

    coord.request_stop()
    coord.join(threads, stop_grace_period_secs=10)
    return softmaxmat_r, images_r, labels_r, img_raw_r, superLabels_r


def evaluate():
  """Eval CIFAR-10 for a number of steps."""
  with tf.Graph().as_default() as g:
    # Get images and labels for CIFAR-10.
    eval_data = FLAGS.eval_data == 'test'
    images, img_raw, labels, superLabels = cifar10.inputs(eval_data=eval_data, raw=True, batch=testSize)

    # Build a Graph that computes the logits predictions from the
    # inference model.
    _, _, softmaxMat = cifar10.inference(images, testSize, testKeep=1.0)
    #labels = tf.reshape(tf.one_hot(labels,12,dtype='int32'),(-1,2,6))
    #flatSoftmaxMat = tf.reshape(softmaxMat,(-1,12))
    # Calculate predictions.
    #top_k_op = tf.nn.in_top_k(flatSoftmaxMat, labels, 1)

    # Restore the moving average version of the learned variables for eval.
    variable_averages = tf.train.ExponentialMovingAverage(
        cifar10.MOVING_AVERAGE_DECAY)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)

    # Build the summary operation based on the TF collection of Summaries.
    summary_op = tf.summary.merge_all()

    summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)

    while True:
      softmaxmat_r, images_r, labels_r, img_raw_r, superLabels_r = eval_once(saver, summary_writer, summary_op, softmaxMat, images, labels, img_raw, superLabels)
      if FLAGS.run_once:
        break
      time.sleep(FLAGS.eval_interval_secs)
  return softmaxmat_r, images_r, labels_r, img_raw_r, superLabels_r

In [3]:
softmaxmat, img, lbls, img_raw, superlbls = evaluate()

Tensor("batch_1:0", shape=(2560, 24, 24, 3), dtype=float32)


NotFoundError: Key l_acol/biases2/ExponentialMovingAverage not found in checkpoint
	 [[Node: save/RestoreV2_6 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_6/tensor_names, save/RestoreV2_6/shape_and_slices)]]

Caused by op u'save/RestoreV2_6', defined at:
  File "/opt/conda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/conda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/opt/conda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/opt/conda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in s

In [ ]:
softmaxmat.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
bgr2rgb = np.array([0,1,2])
imgs_raw = img_raw[:,:,:,bgr2rgb]

In [ ]:
ind = 6
plt.imshow(imgs_raw[ind].astype(np.uint8),vmin=0,vmax=255,aspect='equal', interpolation='none') 
labelNames[lbls[ind]]

In [ ]:
superClass = np.argmax(np.max(softmaxmat, axis=2),axis=1)

In [ ]:
cluster = []
for i in range(testSize):
    cluster.append(softmaxmat[i,superClass[i],:])
cluster = np.argmax(np.array(cluster),axis=1)

In [ ]:
superClass

In [ ]:
classif = pd.DataFrame([classNames[superClass],cluster,range(testSize)],['superClass','cluster','index']).T
classifGrouped = classif.groupby(['superClass','cluster']).count()
classifGrouped

In [ ]:
#plot 5 images of each cluster in class c:
c='Stuff and Horse'
#c='Animals'

In [ ]:
t=[classif[(classif['superClass']==c) & (classif['cluster']==i)]['index'] for i in range(clustCount)]

In [ ]:
imCount=10
f, plots = plt.subplots(len(t),imCount)
f.set_figwidth(10*7)
f.set_figheight(clustCount*7)
plt.tight_layout()
for clust in range(len(t)):
    for index in range(imCount):
        if index < len(t[clust]):
            plots[clust,index].set_title(labelNames[lbls[t[clust].iloc[index]]])
            if len(imgs_raw[t[clust]].shape)>3:
                plots[clust,index].imshow(imgs_raw[t[clust]][index,:,:,:].astype(np.uint8),vmin=0,vmax=255,aspect='equal', interpolation='none')
            else:
                plots[clust,index].imshow(imgs_raw[t[clust]][:,:,:].astype(np.uint8),vmin=0,vmax=255,aspect='equal', interpolation='none')

## Performance scores

In [ ]:
ypred = softmaxmat
onehotLabels = np.zeros((len(lbls),np.max(lbls)+1))
onehotLabels[np.arange(len(lbls)),lbls] = 1
ypred = ypred.reshape((testSize,clustCount*classCount))
#print(np.argmax(ypred,1))
ypred = np.argmax(ypred,1)
print(np.sum(onehotLabels[ypred==1],0))
ylookup = np.argmax([np.sum(onehotLabels[ypred==i],0) for i in range(clustCount*classCount)],1)
yconverted = [ylookup[i] for i in ypred]
correct_prediction = np.equal(yconverted, lbls)
clustAcc = np.mean(correct_prediction)
print('Clustering Accuracy: %g'%(clustAcc))
print(np.reshape(ylookup,(classCount,clustCount)))

In [ ]:
clustcls = np.array(zip(ypred,lbls))
counts = [np.bincount(clustcls[:,1][clustcls[:,0]==i]) for i in range(clustCount*classCount)]
#Per clust "accuracy":
clustAcc = [[np.max(c)/np.sum(c),np.sum(c),labelNames[np.argmax(c)]] if len(c)>0 else 0 for c in counts]
clustAcc[:clustCount]

In [ ]:
clustAcc[clustCount:]

In [ ]:
pur = np.sum([np.max(c) if len(c)>0 else 0 for c in counts])/testSize
print("Purity of clustering: %g"%pur)

In [ ]:
superypred = np.argmax(np.max(softmaxmat,2),1)
correct_prediction = np.equal(superypred, superlbls)
acc = np.mean(correct_prediction)
print('Superclass Accuracy: %g'%(acc))

# TODO
Why do my images look botched?
- Because they are floats, which imshow doesn't like, so it takes only the decimal bit and uses that

In [ ]:
#TSNE raw CIFAR
from sklearn.manifold import TSNE
model = TSNE(verbose=2)
#print(tf.reshape(imgs_raw,[imgs_raw.shape[0],-1]).shape)
tsned = model.fit_transform(np.reshape(imgs_raw,[imgs_raw.shape[0],-1]))

In [ ]:
#print(tsned.shape)
plt.figure(figsize=[50,50])
for im in range(imgs_raw.shape[0]):
    plt.imshow(imgs_raw[im].astype(np.uint8),vmin=0,vmax=255,aspect='equal', interpolation='none',extent=[tsned[im,0]-1,tsned[im,0]+1,tsned[im,1]-1,tsned[im,1]+1])
plt.scatter(tsned[:,0],tsned[:,1])

In [ ]:
plt.figure(figsize=[10,10])
for i in np.unique(lbls):
    plt.scatter(tsned[[lbls==i]][:,0],tsned[[lbls==i]][:,1],cmap='viridis',label=labelNames[i])
plt.legend()